ArcLayer
========

Map of commutes to work within a segment of downtown San Francisco using a deck.gl ArcLayer.

Green indicates a start point, and red indicates the destination.

The data is collected by the US Census Bureau and viewable in the 2017 LODES data set: https://lehd.ces.census.gov/data/

Mapbox requires an API key
command + scroll to rotate map (different on Windows and other combinations work also - shift + scroll)

PATTERN TO CREATE MAPS
data
view (location, zoom level, etc.)
layer
render map
display and save map (to_html(), show())

In [1]:
import mapboxgl
mapboxgl.__version__

'0.10.2'

In [2]:
import os
from dotenv import load_dotenv
# .env file to environment
#load_dotenv()
load_dotenv(verbose=True)

token = os.getenv('MAPBOX_API_KEY')

!jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
!jupyter nbextension enable --sys-prefix --py pydeck

In [3]:
import pydeck as pdk
import pandas as pd

DATA_URL = "https://raw.githubusercontent.com/ajduberstein/sf_public_data/master/bay_area_commute_routes.csv"

In [4]:
# A bounding box for downtown San Francisco, to help filter this commuter data
DOWNTOWN_BOUNDING_BOX = [
    -122.43135291617365,
    37.766492914983864,
    -122.38706428091974,
    37.80583561830737,
]


def in_bounding_box(point):
    """Determine whether a point is in our downtown bounding box"""
    lng, lat = point
    in_lng_bounds = DOWNTOWN_BOUNDING_BOX[0] <= lng <= DOWNTOWN_BOUNDING_BOX[2]
    in_lat_bounds = DOWNTOWN_BOUNDING_BOX[1] <= lat <= DOWNTOWN_BOUNDING_BOX[3]
    return in_lng_bounds and in_lat_bounds

In [5]:
df = pd.read_csv(DATA_URL)
# Filter to bounding box
df = df[df[["lng_w", "lat_w"]].apply(lambda row: in_bounding_box(row), axis=1)]

In [6]:
df.head()

,S000,w_geocode,h_geocode,lat_w,lng_w,geoid10,lat_h,lng_h
1,2,60750615001027,60014004001009,37.788996,-122.399133,60014004001009,37.849728,-122.253480
7,2,60750117002022,60014011003009,37.787461,-122.404334,60014011003009,37.825330,-122.263653
8,5,60750176014001,60014011003009,37.776614,-122.416391,60014011003009,37.825330,-122.263653
9,2,60750615001008,60014011003009,37.791575,-122.393733,60014011003009,37.825330,-122.263653
10,2,60750615001016,60014011003009,37.789900,-122.399964,60014011003009,37.825330,-122.263653


df = df[['lat_w', 'lng_w', 'lat_h', 'lng_h']]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36526 entries, 1 to 189690
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   S000       36526 non-null  int64  
 1   w_geocode  36526 non-null  int64  
 2   h_geocode  36526 non-null  int64  
 3   lat_w      36526 non-null  float64
 4   lng_w      36526 non-null  float64
 5   geoid10    36526 non-null  int64  
 6   lat_h      36526 non-null  float64
 7   lng_h      36526 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 2.5 MB


df = df.loc[df.geoid10==60014004001009]
df

In [8]:
GREEN_RGB = [0, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data=df,
    get_width="S000 * 2",
    get_source_position=["lng_h", "lat_h"],
    get_target_position=["lng_w", "lat_w"],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(
    latitude=37.7576171,
    longitude=-122.5776844,
    bearing=45,
    pitch=50,
    zoom=8,
)


TOOLTIP_TEXT = {"html": "{S000} jobs <br /> Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT)
#r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=True)

In [9]:
#r.to_html("arc_layer.html")
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": 45, "latitude": 37.7576171, "l…

In [10]:
GREAT_CIRCLE_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/flights.json"  # noqa
df = pd.read_json(GREAT_CIRCLE_LAYER_DATA)
df.head()

,from,to
0,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Liverpool John Lenn..."
1,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Newcastle Int'l', '..."
2,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Madinah Int'l', 'ab..."
3,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Sakirpasa', 'abbrev..."
4,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Amarillo Int'l', 'a..."


In [11]:
# Use pandas to prepare data for tooltip
df["from_name"] = df["from"].apply(lambda f: f["name"])
df["to_name"] = df["to"].apply(lambda t: t["name"])

In [12]:
df.head()

,from,to,from_name,to_name
0,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Liverpool John Lenn...",San Francisco Int'l,Liverpool John Lennon
1,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Newcastle Int'l', '...",San Francisco Int'l,Newcastle Int'l
2,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Madinah Int'l', 'ab...",San Francisco Int'l,Madinah Int'l
3,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Sakirpasa', 'abbrev...",San Francisco Int'l,Sakirpasa
4,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Amarillo Int'l', 'a...",San Francisco Int'l,Amarillo Int'l


In [18]:
df['from'][0]

{'type': 'major',
 'name': "San Francisco Int'l",
 'abbrev': 'SFO',
 'coordinates': [-122.38347034444931, 37.61702508680534]}

In [13]:
# Define a layer to display on a map
layer = pdk.Layer(
    "GreatCircleLayer",
    df,
    pickable=True,
    get_stroke_width=12,
    get_source_position="from.coordinates",
    get_target_position="to.coordinates",
    get_source_color=[64, 255, 0],
    get_target_color=[0, 128, 200],
    auto_highlight=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=50, longitude=-40, zoom=1, bearing=0, pitch=0)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{from_name} to {to_name}"},
)

r.picking_radius = 10

In [14]:
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": 0, "latitude": 50, "longitude"…